## Start VLM server

In [1]:
from unsloth.dataprep import SyntheticDataKit
import os
import sys
import requests

# Add vllm directory to PATH (important if vllm is installed in a custom location)
vllm_path = "/work/users/s/m/smerrill/.conda/envs/synthetic-data/bin"
os.environ["PATH"] = vllm_path + os.pathsep + os.environ["PATH"]

# Optional: Add Hugging Face cache if needed
os.environ["HF_HOME"] = "/work/users/s/m/smerrill/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/work/users/s/m/smerrill/.cache/huggingface"

generator = SyntheticDataKit.from_pretrained(
    # Choose any model from https://huggingface.co/unsloth
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Longer sequence lengths will be slower!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/nas/longleaf/home/smerrill/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-21 16:38:34 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-21 16:38:34 [__init__.py:239] Automatically detected platform cuda.


2025-05-21 16:38:36,469	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Unsloth: Your GPU cannot handle sequence lengths of 256 due to limited GPU memory.
Unsloth: Your GPU can only handle approximately the maximum sequence length of 256.
Unsloth: Using dtype = torch.bfloat16 for vLLM.
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 7.88%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.49 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 256. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.0 GB. Also swap space = 6 GB.
vLLM STDOUT: INFO 05-21 16:38:47 [__init__.py:239] Automatically detected platform cuda.
vLLM STDOUT: INFO 05-21 16:38:55 [api_server.py:1043] vLLM API server version 0.8.5.post1
vLLM STDOUT: INFO 05-21 16:38:55 [api_server.py:1044] args: Namespace(subparser='serve', model_tag='unsloth/Llama-3.2-3B-Instruct', config='', host=None, port=8000, uvicorn_log_level='info', disable_uvicorn_access_log=False, allow_credentials=False, allowed_origins=['*'], allowed_meth

## Generate QA Pairs + Auto clean data

In [2]:
generator.prepare_qa_generation(
    output_folder = "data", # Output location of synthetic data
    temperature = 0.7, # Higher temp makes more diverse datases
    top_p = 0.95,
    overlap = 64, # Overlap portion during chunking
    max_generation_tokens = 512, # Can increase for longer QA pairs
)

### Sanity Checks

In [3]:
!synthetic-data-kit system-check

 VLLM server is running at http://localhost:8000/v1
Available models: {'object': 'list', 'data': [{'id': 
'unsloth/Llama-3.2-3B-Instruct', 'object': 'model', 'created': 1747859970, 
'owned_by': 'vllm', 'root': 'unsloth/Llama-3.2-3B-Instruct', 'parent': None, 
'max_model_len': 2048, 'permission': [{'id': 
'modelperm-e1b76278cdc74aae9a2fb06d336c0654', 'object': 'model_permission', 
'created': 1747859970, 'allow_create_engine': False, 'allow_sampling': True, 
'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 
'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': 
False}]}]}
⠋ Checking VLLM server at http://localhost:8000/v1...


In [5]:
url = "http://localhost:8000/v1/chat/completions"

payload = {
    "model": "unsloth/Llama-3.2-3B-Instruct",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    "temperature": 0.7,
    "max_tokens": 100
}

response = requests.post(url, json=payload)

# Pretty-print the result
if response.status_code == 200:
    print(response.json()["choices"][0]["message"]["content"])
else:
    print(f"Error {response.status_code}: {response.text}")


The capital of France is Paris.


### Ingest

In [7]:
!synthetic-data-kit ingest /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/acuff.pdf
!synthetic-data-kit ingest /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/le.pdf
!synthetic-data-kit ingest /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/paige.pdf
!synthetic-data-kit ingest /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/osborne.pdf

!synthetic-data-kit ingest /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/YT_acuff.pdf
!synthetic-data-kit ingest /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/YT_le.pdf
!synthetic-data-kit ingest /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/YT_paige.pdf
!synthetic-data-kit ingest /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/YT_osborne.pdf

⠙ Processing /nas/longleaf/home/smerrill/notebooks/LLM/pdfs/acuff.pdf.....
 Text successfully extracted to data/output/acuff.txt


### Generate

#### Custom Bios text

In [9]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /nas/longleaf/home/smerrill/notebooks/LLM/data/output/acuff.txt  \
        --num-pairs 25 \
        --type "qa"
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /nas/longleaf/home/smerrill/notebooks/LLM/data/output/le.txt  \
        --num-pairs 25 \
        --type "qa"

⠋ Generating qa content from 
⠙ Generating qa content from put/acuff.txt...
⠹ Generating qa content from put/acuff.txt...
⠸ Generating qa content from put/acuff.txt...
⠼ Generating qa content from put/acuff.txt...
⠴ Generating qa content from put/acuff.txt...
⠦ Generating qa content from put/acuff.txt...
⠧ Generating qa content from put/acuff.txt...
⠇ Generating qa content from put/acuff.txt...
⠏ Generating qa content from put/acuff.txt...
⠋ Generating qa content from put/acuff.txt...
⠙ Generating qa content from put/acuff.txt...
⠹ Generating qa content from put/acuff.txt...
⠸ Generating qa content from put/acuff.txt...
⠼ Generating qa content from put/acuff.txt...
Processing 2 chunks to generate QA pairs...cuff.txt...
⠼ Generating qa content from 
⠴ Generating qa content from put/acuff.txt...
⠦ Generating qa content from put/acuff.txt...
⠧ Generating qa content from put/acuff.txt...
⠇ Generating qa content from put/acuff.txt...
⠏ Generating qa content from put/acuff.txt...
⠋ Generatin

In [10]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /nas/longleaf/home/smerrill/notebooks/LLM/data/output/paige.txt  \
        --num-pairs 25 \
        --type "qa"
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /nas/longleaf/home/smerrill/notebooks/LLM/data/output/osborne.txt  \
        --num-pairs 25 \
        --type "qa"

⠋ Generating qa content from 
⠙ Generating qa content from put/paige.txt...
⠹ Generating qa content from put/paige.txt...
⠸ Generating qa content from put/paige.txt...
⠼ Generating qa content from put/paige.txt...
⠴ Generating qa content from put/paige.txt...
⠦ Generating qa content from put/paige.txt...
⠧ Generating qa content from put/paige.txt...
⠇ Generating qa content from put/paige.txt...
⠏ Generating qa content from put/paige.txt...
⠋ Generating qa content from put/paige.txt...
⠙ Generating qa content from put/paige.txt...
⠹ Generating qa content from put/paige.txt...
Processing 2 chunks to generate QA pairs...aige.txt...
⠹ Generating qa content from 
⠸ Generating qa content from put/paige.txt...
⠼ Generating qa content from put/paige.txt...
⠴ Generating qa content from put/paige.txt...
⠦ Generating qa content from put/paige.txt...
⠧ Generating qa content from put/paige.txt...
⠇ Generating qa content from put/paige.txt...
⠏ Generating qa content from put/paige.txt...
⠋ Generatin

### Youtube Transcripts

In [5]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /nas/longleaf/home/smerrill/notebooks/LLM/data/output/YT_acuff.txt  \
        --num-pairs 25 \
        --type "qa"
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /nas/longleaf/home/smerrill/notebooks/LLM/data/output/YT_le.txt  \
        --num-pairs 25 \
        --type "qa"

⠋ Generating qa content from 
⠙ Generating qa content from put/YT_acuff.txt...
⠹ Generating qa content from put/YT_acuff.txt...
⠸ Generating qa content from put/YT_acuff.txt...
⠼ Generating qa content from put/YT_acuff.txt...
⠴ Generating qa content from put/YT_acuff.txt...
⠦ Generating qa content from put/YT_acuff.txt...
⠧ Generating qa content from put/YT_acuff.txt...
⠇ Generating qa content from put/YT_acuff.txt...
⠏ Generating qa content from put/YT_acuff.txt...
⠋ Generating qa content from put/YT_acuff.txt...
⠙ Generating qa content from put/YT_acuff.txt...
⠹ Generating qa content from put/YT_acuff.txt...
Processing 3 chunks to generate QA pairs...T_acuff.txt...
⠹ Generating qa content from 
⠸ Generating qa content from put/YT_acuff.txt...
⠼ Generating qa content from put/YT_acuff.txt...
⠴ Generating qa content from put/YT_acuff.txt...
⠦ Generating qa content from put/YT_acuff.txt...
⠧ Generating qa content from put/YT_acuff.txt...
⠇ Generating qa content from put/YT_acuff.txt...


In [4]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /nas/longleaf/home/smerrill/notebooks/LLM/data/output/YT_paige.txt  \
        --num-pairs 25 \
        --type "qa"
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /nas/longleaf/home/smerrill/notebooks/LLM/data/output/YT_osborne.txt  \
        --num-pairs 25 \
        --type "qa"

⠋ Generating qa content from 
⠙ Generating qa content from put/YT_paige.txt...
⠹ Generating qa content from put/YT_paige.txt...
⠸ Generating qa content from put/YT_paige.txt...
⠼ Generating qa content from put/YT_paige.txt...
⠴ Generating qa content from put/YT_paige.txt...
⠦ Generating qa content from put/YT_paige.txt...
⠧ Generating qa content from put/YT_paige.txt...
⠇ Generating qa content from put/YT_paige.txt...
⠏ Generating qa content from put/YT_paige.txt...
⠋ Generating qa content from put/YT_paige.txt...
⠙ Generating qa content from put/YT_paige.txt...
⠹ Generating qa content from put/YT_paige.txt...
⠸ Generating qa content from put/YT_paige.txt...
Processing 2 chunks to generate QA pairs...T_paige.txt...
⠼ Generating qa content from 
⠼ Generating qa content from put/YT_paige.txt...
⠴ Generating qa content from put/YT_paige.txt...
⠦ Generating qa content from put/YT_paige.txt...
⠧ Generating qa content from put/YT_paige.txt...
⠇ Generating qa content from put/YT_paige.txt...


### Convert to HF format

In [9]:
generated_path ='/nas/longleaf/home/smerrill/notebooks/LLM/data/generated'
qa_pairs_filenames = os.listdir(generated_path)
for filename in qa_pairs_filenames:
    filename = os.path.join(generated_path, filename)
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

⠋ Converting 
/nas/longleaf/home/smerrill/notebooks/LLM/data/generated/test_write.json to ft 
format with json storage...
L Error: Unrecognized data format - expected QA pairs or conversations
⠋ Converting 
/nas/longleaf/home/smerrill/notebooks/LLM/data/generated/acuff_qa_pairs.json to 
ft format with json storage...
 Converted to ft format and saved to data/final/acuff_qa_pairs_ft.json
⠋ Converting 
/nas/longleaf/home/smerrill/notebooks/LLM/data/generated/le_qa_pairs.json to ft 
format with json storage...
 Converted to ft format and saved to data/final/le_qa_pairs_ft.json
⠋ Converting 
/nas/longleaf/home/smerrill/notebooks/LLM/data/generated/paige_qa_pairs.json to 
ft format with json storage...
 Converted to ft format and saved to data/final/paige_qa_pairs_ft.json
⠋ Converting 
/nas/longleaf/home/smerrill/notebooks/LLM/data/generated/osborne_qa_pairs.json 
to ft format with json storage...
 Converted to ft format and saved to data/final/osborne_qa_pairs_ft.json
⠋ Converting 
/nas/lo

In [ ]:
#generator.cleanup()

Attempting to terminate the VLLM server gracefully...
Server terminated gracefully.
